# HW3

In this homeowork, we will
- Build a neural network based part of speech (POS) tagger [35 points].
- Run the model on a GPU at Google Colab [10].
- Extend the model to use pretrained embeddings [15].
- Extend the model in some manner of your own choosing [30].

There are a total of 8 tasks.

Acknowledgements: Part of this homework were adapted from work done by
[Ben Trevett](https://github.com/bentrevett) and
[Kevin Gimpel](https://home.ttic.edu/~kgimpel/).

### Installing libraries

This homework requires you to install the following libraries, which are the current latest versions.  They are not, however, the default versions installed by, say, Anaconda.  The code has been tested on these versions; but it is possible it works on other versions as well.

- PyTorch 1.11.0
- torchtext 0.12.0
- TorchData 0.3.0

You may need to read some of the documentation for these libraries to solve this homework.

Since these libraries are evolving rapidly, you may get some warnings.  Please ignore them unless the code later breaks down.

When working with colab, in a new runtime, you'll have to install the above libraries. (Once installed you usually do not need reinstallation if you, e.g., restart the runtime.)  One way to install them is via the following commands (they automatically update PyTorch).

`!pip install torchtext==0.12.0`

`!pip install torchdata==0.3.0`

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext import data
from torchtext import datasets

from collections import Counter
from torchtext.vocab import vocab

import numpy as np
import random

In [2]:
# Check the library versions
torch.__version__, torchtext.__version__

('1.11.0', '0.12.0')

In [3]:
SEED = 30255 # Specify a seed for reproducability

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(53113)
torch.backends.cudnn.deterministic = True

### Using GPUs

We want our code to run on a CPU or on a [GPU](https://en.wikipedia.org/wiki/Graphics_processing_unit) when available, as on Google Colab.  The code will likely run at least five times as fast on a GPU. PyTorch connects with a GPU via the system interface [CUDA](https://en.wikipedia.org/wiki/CUDA).

The following tells us whether CUDA/GPU is available or not.  The changes you have to make so that your code can use a GPU when available are surprisingly small. All such changes have already been made in the provided code below. They are:
- the data loader needs to place the input to the model appropriately using `to(device).`
- similarly, the model object itself needs to be placed appropriately using `to(device).`


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### Cache for GloVe

If you have already downloaded GloVe in a directory, you can reuse it.  Please change the following if so.

In [ ]:
if device == 'cpu':
    VECTORS_CACHE_DIR = '/Users/amitabh/.vector_cache'
    # Please change above to your cache
else:
    VECTORS_CACHE_DIR = './.vector_cache'
    # This is the default cache on Colab. Caching may not work
    # as expected on Colab.

### UDPOS Dataset

We'll work with the [`UDPOS`](https://pytorch.org/text/stable/datasets.html#udpos) dataset included with torchtext.  It contains about 12,500 sentences and the POS tags for the each word.  There are, in fact, two sets of tags, based on two different POS tagging standards: [Universal Dependency](https://universaldependencies.org/u/pos/) (UD) and [Penn Treebank](https://www.sketchengine.eu/penn-treebank-tagset/) (PTB). It has additionally about 2000 sentences in the validation and test sets.  The words and corresonding tags are organized into lists.

In [5]:
train_iter = datasets.UDPOS(split = 'train')

for i, example in enumerate(train_iter):
    print(f'The sentence for Example {i} ---')
    print(example[0])
    print(f'The UD tags for Example {i} ---')
    print(example[1])
    print(f'The PTB tags for Example {i} ---')
    print(example[2])
    print()
    if i == 2: break

The sentence for Example 0 ---
['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.']
The UD tags for Example 0 ---
['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']
The PTB tags for Example 0 ---
['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']

The sentence for Example 1 ---
['[', 'This', 'killing', 'of', 'a', 'respected', 'cleric', 'will', 'be', 'causing', 'us', 'trouble', 'for', 'years', 'to', 'come', '.', ']']
The UD tags for Example 1 ---
['PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'AUX',

/Users/amitabh/miniforge3/lib/python3.9/site-packages/torch/utils/data/datapipes/utils/common.py:24: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn(
/Users/amitabh/miniforge3/lib/python3.9/site-packages/torch/utils/data/datapipes/iter/selecting.py:54: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn("Lambda function is not supported for pickle, please use "


### Preprocessing

In our neural network-based model we can only accept fixed sized inputs.  Each input corresponds to a subsequence, consisting of a  center word and a number $w$ of words before and after, and the label is the POS tag for the center word. We often refer to the subsequence as a window of size $(2w + 1)$.

So given a sentence, say, "This killing of a respected cleric.' with, tags, say 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT' (the last tag is for the period at the end of the sentence), some of our input examples for the model are the following (for $w = 1$):
- "This killing of", 'NOUN'
- "killing of a", 'ADP'
- "respected cleric.", 'NOUN'.

But the above scheme would imply that, when $w = 1$, the tags of the first and last words for any sentence would never be part of an example.  To avoid that, we add dummy '\<s>' and '\</s>' words, and corresponding tags, to the beginning and end of each sentence.  We need to add as many dummy words as $w$.  So if $w = 2$, we first convert the above sentence and its tags to
- "\<s> \<s> This killing of a respected cleric. \</s> \</s>' 
- 'STAG', 'STAG','DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'ETAG', 'ETAG'. 

**Task 1** [5 points]: Most of the code for preprocessing is given below.  But some parts are missing.  Complete the code.

In [ ]:
W = 1
WINDOW_SIZE = (2 * W + 1)

SENT_START_WORD = '<s>'
SENT_END_WORD = '</s>'
SENT_START_TAG = '<STAG>'
SENT_END_TAG = '<ETAG>'


def add_sent_start_end(data_iter, w):
    for (words, ud_tags, ptb_tags) in data_iter:
        new_words = [SENT_START_WORD] * w + words + [SENT_END_WORD] * w
        new_ud_tags = [SENT_START_TAG] * w + ud_tags + [SENT_END_TAG] * w
        ## MISSING PART: ADD YOUR CODE BELOW
       
        ## ADD YOUR CODE ABOVE
        yield(new_words, new_ud_tags, new_ptb_tags)
        
def create_windows(data_iter, w):
    window_size = 2*w + 1
    for (words, ud_tags, ptb_tags) in data_iter:
        words_zip = zip(*[words[i:] for i in range(window_size)])
        ud_zip = zip(*[ud_tags[i:] for i in range(window_size)])
        ## MISSING PART: ADD YOUR CODE BELOW
       
        ## ADD YOUR CODE ABOVE
        for word_sseq, ud_sseq, ptb_sseq in zip(
                words_zip, ud_zip, ptb_zip):
            yield(word_sseq, ud_sseq, ptb_sseq)
            
def preprocess_data_seq(data_iter, w):
    ## MISSING PART: ADD YOUR CODE BELOW
    pass

def test_preprocess_data_seq():
    
    # WARNING: The following test assumes a particular default
    # sequence of examples in the PyTorch UDPOS dataset. If you
    # suspect the sequence is different for your dataset, please
    # adapt the test.

    train_iter_0 = datasets.UDPOS(split = 'train')    
    train_iter_demo = preprocess_data_seq(train_iter_0, 1)
    ex0 = (('<s>', 'Al', '-'), 
           ('<STAG>', 'PROPN', 'PUNCT'), 
           ('<STAG>', 'NNP', 'HYPH'))
    ex1 = (('Al', '-', 'Zaman'), 
           ('PROPN', 'PUNCT', 'PROPN'), 
           ('NNP', 'HYPH', 'NNP'))
    ex2 = (('-', 'Zaman', ':'), 
           ('PUNCT', 'PROPN', 'PUNCT'), 
           ('HYPH', 'NNP', ':'))
    assert ex0 == next(train_iter_demo)
    assert ex1 == next(train_iter_demo)
    assert ex2 == next(train_iter_demo)
    print('Test passed.')
    
test_preprocess_data_seq()

### Create vocabularies for words and tags

Since the number of tags is reasonably large, we use the `Vocab` class to create objects `vocab_words`, `vocab_ud`, and `vocab_ptb` for both words and the two sets of tags from the training set.

In [ ]:
# It is important to recreate the training set afresh 
# each time they are used, since they are Python iterators, and
# once use, cannot be reused.  One could, however, cast them into
# a list, which would store them permanently.

train_iter_0 = datasets.UDPOS(split = 'train')    
train_iter_vocab = preprocess_data_seq(train_iter_0, 1)

counter_words = Counter()
counter_ud = Counter()
counter_ptb = Counter()
for (text, pos_ud, pos_ptb) in train_iter_vocab:
    counter_words.update(text)
    counter_ud.update(pos_ud)
    counter_ptb.update(pos_ptb)
vocab_words = vocab(counter_words,  specials = ['<unk>'], 
                    special_first = True)    
vocab_words.set_default_index(0)
vocab_ud = vocab(counter_ud)
vocab_ptb = vocab(counter_ptb)

Observe that we set a special word for vocab_words, but none for vocab_ud or vocab_ptb.  Why is that? (You don't have to submit an answer.)

### Collate function for customized data loader

**Task 2** [10]: Write a collate function that takes a batch of examples, and for a given window width and tag type, returns the following
- a tensor corresponding to the index of the tag for the center word in each example of the batch. Recall, each example is a window with a width of $(2w + 1)$ words.
- a tensor corresponding to the index of each of the words in the example, according the to vocabulary for words.

Note: The collate function is called by the data loader with the batch of examples from the data set.  So any additional parameters to the function should have default values.

From this task onwards we'll only work with the UD tags.

In [ ]:
from torch.utils.data import DataLoader

TAG = 'ud'

def collate_fn(batch, w = W, tag = TAG):
    
    ## WRITE YOUR CODE BELOW
    
    ## WRITE YOUR CODE ABOVE
    # The tensors you return should be placed in the correct device
    # as shown below.
    return labels.to(device), word_idxs.to(device)


def test_collate():
    
    pos = [5, 6, 1, 4]  
    examples = []
    for perm in ['03022', '33210', '33211', '11101']:
        words = []
        utags = []
        ptags = []
        for ind in perm:
            ind = int(ind)
            words.append(vocab_words.lookup_token(pos[ind]))
            utags.append(vocab_ud.lookup_token(pos[ind]+1))
            ptags.append(vocab_ptb.lookup_token(pos[ind]))
        examples.append((words, utags, ptags))
    lt =  torch.tensor([5, 1, 1, 6]).to(device)
    lt =  torch.tensor([6, 2, 2, 7]).to(device)
    wt = torch.tensor([
        [5, 4, 5, 1, 1],
        [4, 4, 1, 6, 5],
        [4, 4, 1, 6, 6],
        [6, 6, 6, 5, 6]]).to(device)
    rlt, rwt = collate_fn(examples, w = 2)
    assert torch.equal(lt, rlt)
    assert torch.equal(wt, rwt)
    print('test_collate passed')

test_collate()
    

### Writing the class for the neural network model

**Task 3** [15]: Write a class that implements the following neural network for input $(w_1, w_2, w_3)$ and label $y$ corresponding to tag of $w_2$. (Also discussed in class on April 15, 2021).
$$
\begin{eqnarray*}
x &=& [E_{[w_1]}, E_{[w_2]}, E_{[w_3]}] \\
h &=& \text{tanh}(x W^1 + b^1)\\
\tilde{y} & = & hW^2 + b^2\\
\hat{y} & = & \text{softmax}(y)\\
\end{eqnarray*}
$$
The model returns $\ln \hat{y}$, and expects the training loop to use `nn.NLLLoss()` to compute the cross entropy loss, given $y$. Assume the embedding has vectors of size $300$, and $h$ is a vector of size $128$.  Please read the documentation for `nn.Embedding(),` `nn.Linear(),`, and `nn.Tanh()` to learn how to use them.

*Hint:* Unlike HW2, here the input (for each example in a batch) consist of $(2w+1)$ numbers corresponding to the indices of the words in the window. These are converted to $(2w+1)$ vectors when they are passed to the embedding layer. They have to be concatenated into a single vector of length $(2w+1) \times 300$ before they are sent to the first linear layer ($W^1$).  Use `reshape()` or `view()` in the `forward()` function below to achieve this.


In [ ]:
class NNPOSTagger(nn.Module):
    def __init__(self,
                 window_size,
                 vocab_size, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 nonlinearity, 
                 # These are used for later tasks
                 use_glove = False, 
                 freeze_glove = False):      
        super(NNPOSTagger, self).__init__()
        
        ## WRITE YOUR CODE BELOW
        
        pass
      
        
    def forward(self, word_idxs_batch):
        
        ## WRITE YOUR CODE BELOW.
        
        pass

#### Instantiating the model

PyTorch uses `to(device)` to specify whether the model is to be used with a CPU or with a GPU/CUDA.

In [ ]:
model = NNPOSTagger(window_size = WINDOW_SIZE, 
                    vocab_size = len(vocab_words), 
                     embedding_dim = 300, 
                     hidden_dim = 128, 
                     output_dim = len(vocab_ud),
                     nonlinearity = nn.Tanh(), 
                     use_glove = False,
                     freeze_glove = False).to(device)

### Training an epoch

**Task 4** [5]: Write a function to train one epoch of the dataset. This is similar to the code in HW2.

In [ ]:
loss_function = torch.nn.NLLLoss()

def train_an_epoch(dataloader):
    
    ### WRITE YOUR CODE BELOW
    pass
    

### Computing the accuracy

The function is given below.

In [ ]:
def get_accuracy(dataloader):
    model.eval()
    with torch.no_grad():    
        total_acc, total_count = 0, 0
        for idx, (label, word_idxs) in enumerate(dataloader):
            log_probs = model(word_idxs)
            total_acc += (log_probs.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

### Creating the data loaders

In [ ]:
BATCH_SIZE = 64 # batch size for training
  
train_0, valid_0, test_0 = train_data_0 = datasets.UDPOS(
    split = ('train', 'valid', 'test'))
train_data = list(preprocess_data_seq(train_0, W))
valid_data = list(preprocess_data_seq(valid_0, W))
test_data = list(preprocess_data_seq(test_0, W))

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE,
                              shuffle=True, 
                              collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_data, batch_size=BATCH_SIZE,
                              shuffle=False, 
                              collate_fn=collate_fn)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE,
                             shuffle=False, 
                             collate_fn=collate_fn)

### Training

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline

EPOCHS = 3 # epoch
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

accuracies=[]
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train_an_epoch(train_dataloader)
    accuracy = get_accuracy(valid_dataloader)
    accuracies.append(accuracy)
    time_taken = time.time() - epoch_start_time
    print(f'Epoch: {epoch}, time taken: {time_taken:.1f}s, validation accuracy: {accuracy:.3f}.')
    
plt.plot(range(1, EPOCHS+1), accuracies)

### Running on Google Colab

Google Colab provides GPUs on which we can run Jupyter notebooks. These are for free as long as the job takes less than, say, 4 hours. Our job will take much less.

Please create an account on Google if you don't have one.  Please read a tutorial on using Colab (such as [this](https://colab.research.google.com/notebooks/intro.ipynb)).  Upload your notebook on Colab. **Change the runtime type to GPU.** Make any changes required to get your code to run on Colab. (Normally there should  be no change required.) 

For all following tasks that required running your code for several epochs, we recommend developing your code locally, but finally running it on Colab to get the results.

**Task 5** [10]: Run the model for a sufficient number of epochs (not more than 30) such that the model shows overfitting, if at all, and submit a pdf of the plot of accuracy against number of epochs. Determine the optimal number of epochs to train for. Write code to estimate the accuracy of your model (using the test set) corresponding to this optimal number of epocs and report this estimated accuracy.

### Initializing with pre-trained embeddings

In the above model, the embedding matrix is initialized to either zero or random vectors, but trained along with the rest of the parameters. It may help to initialize the embedding matrix from GloVe vector embeddings. You may then choose to freeze the embedding matrix (i.e., not update the vectors during training) or train them to adapt to the POS examples. (For further details you may optionally read Section 10.1--3 from the Goldberg textbook.)

The following code snippet creates a embedding matrix that has a vector corresponding to each of our `vocab_words` created from the training set.

In [ ]:
from torchtext import vocab

glove = vocab.GloVe('6B',cache=VECTORS_CACHE_DIR)
glove_vectors = glove.get_vecs_by_tokens(vocab_words.get_itos())

**Task 6** [10]: Rewrite/extend the code for the module to initialize the embedding layer with `glove_vectors` created above.  Please read the documentation for the function [`nn.Embedding.from_pretrained()`](https://pytorch.org/docs/master/generated/torch.nn.Embedding.html#torch.nn.Embedding.from_pretrained) to understand how this is done. Submit corresponding plot and report on the performance when the embedding is frozen.

**Task 7** [5]: Submit corresponding plot and report on the performance when the embedding is not frozen.

### Extending the model

**Task 8** [30]: Try three different ways of your choosing to improve the performance of the model. You may want to vary w, or add additional layers to the network, or increase the size of the hidden vectors $h$, or try with different activation functions. Report on the results you get. Do you think your POS tagger is comparable to a human tagger?

This is an open-ended task.  On it you should spend at least half the time as you spend in total on the previous tasks.  